# Анализ базы данных крупного сервиса для чтения книг по подписке

Во время пандемии у людей появилось больше времени на чтение книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать. Компания решила быть на волне и купила крупный сервис для чтения книг по подписке.

## Шаг 1. Подключение к базе данных

In [4]:
#!pip install psycopg2

Defaulting to user installation because normal site-packages is not writeable


In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [3]:
# функция для запросов
def q(query):
    return print(pd.io.sql.read_sql(query, con = engine))

## Шаг 2. Чтение таблиц из базы данных

In [4]:
q('''SELECT * 
     FROM books ''')

     book_id  author_id                                              title  \
0          1        546                                       'Salem's Lot   
1          2        465                 1 000 Places to See Before You Die   
2          3        407  13 Little Blue Envelopes (Little Blue Envelope...   
3          4         82  1491: New Revelations of the Americas Before C...   
4          5        125                                               1776   
..       ...        ...                                                ...   
995      996        571           Wyrd Sisters (Discworld  #6; Witches #2)   
996      997        454                        Xenocide (Ender's Saga  #3)   
997      998        201                                    Year of Wonders   
998      999         94                        You Suck (A Love Story  #2)   
999     1000        509  Zen and the Art of Motorcycle Maintenance: An ...   

     num_pages publication_date  publisher_id  
0          594 

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

In [5]:
q('''SELECT * 
     FROM authors ''')

     author_id                                author
0            1                            A.S. Byatt
1            2        Aesop/Laura Harris/Laura Gibbs
2            3                       Agatha Christie
3            4                         Alan Brennert
4            5              Alan Moore/David   Lloyd
..         ...                                   ...
631        632         William Strunk Jr./E.B. White
632        633                           Zadie Smith
633        634                 Zilpha Keatley Snyder
634        635                    Zora Neale Hurston
635        636  Åsne Seierstad/Ingrid Christopherson

[636 rows x 2 columns]


**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

In [6]:
q('''SELECT * 
     FROM publishers ''')

     publisher_id                          publisher
0               1                                Ace
1               2                           Ace Book
2               3                          Ace Books
3               4                      Ace Hardcover
4               5  Addison Wesley Publishing Company
..            ...                                ...
335           336         Workman Publishing Company
336           337                         Wyatt Book
337           338              Yale University Press
338           339                           Yearling
339           340                     Yearling Books

[340 rows x 2 columns]


**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

In [7]:
q('''SELECT * 
     FROM ratings ''')

      rating_id  book_id        username  rating
0             1        1      ryanfranco       4
1             2        1   grantpatricia       2
2             3        1    brandtandrea       5
3             4        2        lorichen       3
4             5        2     mariokeller       2
...         ...      ...             ...     ...
6451       6452     1000  carolrodriguez       4
6452       6453     1000         wendy18       4
6453       6454     1000      jarvispaul       5
6454       6455     1000           zross       2
6455       6456     1000         fharris       5

[6456 rows x 4 columns]


**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

In [8]:
q('''SELECT * 
     FROM reviews''')

      review_id  book_id        username  \
0             1        1    brandtandrea   
1             2        1      ryanfranco   
2             3        2        lorichen   
3             4        3   johnsonamanda   
4             5        3     scotttamara   
...         ...      ...             ...   
2788       2789      999      martinadam   
2789       2790     1000         wknight   
2790       2791     1000  carolrodriguez   
2791       2792     1000         wendy18   
2792       2793     1000      jarvispaul   

                                                   text  
0     Mention society tell send professor analysis. ...  
1     Foot glass pretty audience hit themselves. Amo...  
2     Listen treat keep worry. Miss husband tax but ...  
3     Finally month interesting blue could nature cu...  
4     Nation purpose heavy give wait song will. List...  
...                                                 ...  
2788  Later hospital turn easy community. Fact same ...  
2789   

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Шаг 3. Анализ данных

### Сколько книг вышло после 1 января 2000 года;

In [9]:
q(''' SELECT COUNT(book_id)
      FROM books
      WHERE publication_date > '2000-01-01'
  ''')

   count
0    819


In [10]:
q(''' SELECT COUNT(book_id)
      FROM books
      WHERE publication_date >= '2000-01-01'
  ''')

   count
0    821


819 книг в нашей таблице с данными выпущены после 1 января 2000 года.
821 книга влючая 1 января

### Для каждой книги посчитаем количество обзоров и среднюю оценку;

Для этой задачи используем таблицу ratings и reviews

In [12]:
q('''SELECT ratings.book_id AS book_id,
            COUNT(reviews.text) AS count_reviews,
            AVG(ratings.rating) AS avg_rating
      FROM ratings
      LEFT JOIN reviews ON reviews.book_id = ratings.book_id
      GROUP BY ratings.book_id
      ORDER BY ratings.book_id
      ''')

     book_id  count_reviews  avg_rating
0          1              6    3.666667
1          2              2    2.500000
2          3              9    4.666667
3          4              4    4.500000
4          5             24    4.000000
..       ...            ...         ...
995      996              9    3.666667
996      997             15    3.400000
997      998             20    3.200000
998      999              4    4.500000
999     1000             24    3.833333

[1000 rows x 3 columns]


Ни одна книга не осталась без обзоров. Средний рейтинг книг в наших данных начинается с 1.5 и заканчивается 5.0.

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

Для этой задачи будем использовать две таблицы: publisher и books

In [13]:
q(''' SELECT publisher, 
             COUNT(book_id) AS books_count
      FROM publishers AS p
      LEFT JOIN books as b ON p.publisher_id = b.publisher_id
      WHERE num_pages > 50
      GROUP BY publisher
      ORDER BY books_count DESC
      LIMIT 1
  ''')

       publisher  books_count
0  Penguin Books           42


Издательство penguin books выпустило больше всего книг с 50+ страницами

### Определение автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [16]:
q(''' SELECT a.author, 
           ROUND(AVG(a.avg_rating),1) AS avg_rating
     FROM 
        (SELECT author, 
                b.book_id AS id, 
                AVG(rating) AS avg_rating, 
                COUNT(rating) AS count_rating
          FROM authors AS au
          LEFT JOIN books AS b ON b.author_id = au.author_id
          LEFT JOIN ratings AS r ON r.book_id = b.book_id
          GROUP BY author, id
          HAVING COUNT(rating) >= 50) AS a
     GROUP BY author
     ORDER BY avg_rating DESC
      LIMIT 1
        ''')

                       author  avg_rating
0  J.K. Rowling/Mary GrandPré         4.3


Автор с самой высокой средней оценкой - это Роулинг, автор книг по Гарри Поттеру.

### Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [18]:
q(''' SELECT ROUND(AVG(count), 1)
      FROM (SELECT COUNT(text)
            FROM reviews AS r
            LEFT JOIN (SELECT username, 
                              COUNT(rating_id)
                       FROM ratings
                       GROUP BY username) AS ra ON ra.username = r.username
            WHERE count > 50
            GROUP BY r.username) AS count                                  
  ''')

   round
0   24.3


Результат: Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24

## Шаг 4. Вывод

- Книг вышло после 1 января 2000 года - 819;
- Для каждой книги было посчитано количество обзоров и средняя оценка;
- Определили издательство, которое выпустило наибольшее число книг при их толщине более 50 страниц - это "Penguin Books";
- Определили автора с самой высокой средней оценкой книг (с 50 и более оценками) - J.K. Rowling/Mary GrandPré. 
- Средняя оценка ее книг составляет - 4.284 из 5 возможных;
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24

Полученные данные помогут сформулировать предложение для нового продукта